# Using The Odds API for Arbitrage Opportunities

In [1]:
import requests
import pandas as pd
import numpy as np

API_KEY = '0ccdd8b5c3672b187c7c30fcad24a694'

In [2]:
# defining necessary params for Odds API

SPORT = 'basketball_nba'
REGIONS = 'us'
MARKETS = 'h2h' # two outcomes
ODDS_FORMAT = 'decimal'
DATE_FORMAT = 'iso'
BET_SIZE = 10 # defines monetary amount willing to spend

response = requests.get(f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds', 
                       params = {
                           'api_key': API_KEY,
                           'regions': REGIONS,
                           'markets': MARKETS,
                           'oddsFormat': ODDS_FORMAT,
                           'dateFormat': DATE_FORMAT,
                           }
                       ).json()

print(response)

[{'id': 'bcd6fb33fb5797ee8e2713a11ce3a947', 'sport_key': 'basketball_nba', 'sport_title': 'NBA', 'commence_time': '2023-05-01T23:30:00Z', 'home_team': 'Boston Celtics', 'away_team': 'Philadelphia 76ers', 'bookmakers': [{'key': 'draftkings', 'title': 'DraftKings', 'last_update': '2023-05-01T19:51:45Z', 'markets': [{'key': 'h2h', 'last_update': '2023-05-01T19:51:45Z', 'outcomes': [{'name': 'Boston Celtics', 'price': 1.2}, {'name': 'Philadelphia 76ers', 'price': 4.9}]}]}, {'key': 'fanduel', 'title': 'FanDuel', 'last_update': '2023-05-01T19:51:57Z', 'markets': [{'key': 'h2h', 'last_update': '2023-05-01T19:51:57Z', 'outcomes': [{'name': 'Boston Celtics', 'price': 1.2}, {'name': 'Philadelphia 76ers', 'price': 4.85}]}]}, {'key': 'betmgm', 'title': 'BetMGM', 'last_update': '2023-05-01T19:54:19Z', 'markets': [{'key': 'h2h', 'last_update': '2023-05-01T19:54:19Z', 'outcomes': [{'name': 'Boston Celtics', 'price': 1.2}, {'name': 'Philadelphia 76ers', 'price': 4.75}]}]}, {'key': 'barstool', 'title':

In [3]:
# parsing and storing individual sports events

class Event:
    def __init__(self, data):
        self.data = data
        self.key = data['sport_key']
        self.id = data['id']
        
    def find_odds(self):
        outcomes = len(self.data['bookmakers'][0]['markets'][0]['outcomes'])
        self.outcomes = outcomes
        
        top_odds = [[None, None, float('-inf')] for _ in range(outcomes)]
        bookmakers = event.data['bookmakers']
        for i, bookmaker in enumerate(bookmakers):
            for opportunity in range(outcomes):
                bookmaker_current_odds = float(bookmaker['markets'][0]['outcomes'][opportunity]['price'])
                current_top_odds = top_odds[opportunity][2]
                
                if bookmaker_current_odds > current_top_odds:
                    top_odds[opportunity][0] = bookmaker['title']
                    top_odds[opportunity][1] = bookmaker['markets'][0]['outcomes'][opportunity]['name']
                    top_odds[opportunity][2] = bookmaker_current_odds
                    
        self.top_odds = top_odds
        return top_odds
    
    def find_arbitrage(self):
        percent = 0
        for odds in self.top_odds:
            percent += (1.0 / odds[2])
            
        self.percent = percent
        self.earnings = (10 / percent) - 10 # 10 is the number of bets, can change
        
        if percent < 1:
            print("Arbitrage Opportunity")
            return True
        
        print("No Arbitrage Opportunity")
        return False
    
    # have to convert from decimal to american because odds-api doesn't support this
    def convert_items(self):
        top_odds = self.top_odds
        for odds in top_odds:
            val = odds[2]
            if val >= 2:
                return_val = (val - 1) * 100
            elif val < 2:
                return_val = -100 / (val - 1)
            odds[2] = round(return_val, 2)
            
        return top_odds
    
    def calculate_arbitrage(self):
        top_odds = self.top_odds
        arr = []
        for opportunity in range(self.outcomes):
            i_percent = 1 / self.top_odds[opportunity][2]
            bet_amount = (10 * i_percent) / self.percent
            arr.append(round(bet_amount, 2))
            
        self.arr = arr
        return arr
    
    # convert to american from decimal?


In [4]:
events = []
for data in response:
    events.append(Event(data))
    print(data)
    print()
    
arb_events = []
for event in events:
    top_odds = event.find_odds()
    if event.find_arbitrage():
        arb_events.append(event)
        
for event in arb_events:
    event.calculate_arbitrage()
    event.convert_items()

{'id': 'bcd6fb33fb5797ee8e2713a11ce3a947', 'sport_key': 'basketball_nba', 'sport_title': 'NBA', 'commence_time': '2023-05-01T23:30:00Z', 'home_team': 'Boston Celtics', 'away_team': 'Philadelphia 76ers', 'bookmakers': [{'key': 'draftkings', 'title': 'DraftKings', 'last_update': '2023-05-01T19:51:45Z', 'markets': [{'key': 'h2h', 'last_update': '2023-05-01T19:51:45Z', 'outcomes': [{'name': 'Boston Celtics', 'price': 1.2}, {'name': 'Philadelphia 76ers', 'price': 4.9}]}]}, {'key': 'fanduel', 'title': 'FanDuel', 'last_update': '2023-05-01T19:51:57Z', 'markets': [{'key': 'h2h', 'last_update': '2023-05-01T19:51:57Z', 'outcomes': [{'name': 'Boston Celtics', 'price': 1.2}, {'name': 'Philadelphia 76ers', 'price': 4.85}]}]}, {'key': 'betmgm', 'title': 'BetMGM', 'last_update': '2023-05-01T19:54:19Z', 'markets': [{'key': 'h2h', 'last_update': '2023-05-01T19:54:19Z', 'outcomes': [{'name': 'Boston Celtics', 'price': 1.2}, {'name': 'Philadelphia 76ers', 'price': 4.75}]}]}, {'key': 'barstool', 'title': 